In [1]:
import pandas as pd

# Metabolomics counts (count.ps)

In [3]:
df_count = pd.read_excel('Data/han_nature_metabolomics_peak_fc.xlsx', sheet_name='count.ps', engine='openpyxl', index_col=0)
df_count.index.name = 'EntryID'
df_count = df_count.stack().reset_index()
df_count.columns = ['EntryID', 'dname', 'Count']
assert df_count.Count.min()>=0
df_count.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,EntryID,dname,Count
0,3,m_c18p_0003,3.583769e+03
1,3,m_c18p_0009,5.035858e+06
2,3,m_c18p_0012,2.653437e+06
3,3,m_c18p_0014,3.879769e+07
4,3,m_c18p_0015,2.597931e+05


# Metabolomics metadata

In [5]:
df_info = pd.read_excel('Data/han_nature_metabolomics_peak_fc.xlsx', sheet_name='aggregated_md', engine='openpyxl').rename({'Unnamed: 0':'EntryID'}, axis=1)
df_info.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,EntryID,experiment,sample_type,media,subculture_time,preculture_time,c18positive,c18negative,hilicpositive,culture_source,...,grouped_taxonomy,kingdom,phylum,class,order,family,genus,species,strain,morphology
0,3,20181030.0,supernatant,pyg,16.00,30.0,s03081,s03182,s02980,c0082,...,Coprococcus,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,Coprococcus,NaN,HPP0074,NaN
1,4,20190228.0,media_blank,mm,17.25,NaN,s03818,s03658,s03513,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,20190228.0,media_blank,mm,17.25,NaN,s03819,s03659,s03514,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,20190228.0,media_blank,mm,17.25,NaN,s03820,s03660,s03515,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13,20181030.0,supernatant,mm,12.00,30.0,s03029,s03130,s02928,c0206,...,Bacteroides thetaiotaomicron,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,thetaiotaomicron,NaN,NaN


# Join the two tables and select for mega medium

In [6]:
df = pd.merge(df_count, df_info[['experiment','EntryID','sample_type','media']], left_on=['EntryID'], right_on=['EntryID'], how='left')
df = df[(df.sample_type=="media_blank") & (df.media=="mm")] # mega media blank
df = df[df.Count>0] # remove metabolites with zero count
df = df.drop(['EntryID','sample_type',"media"], axis=1)
print("# metabolites in mega medium = %d"%(len(list(set(df.dname)))))
df.head()

# metabolites in mega medium = 556


,dname,Count,experiment
239,m_c18p_0001,1.142775e+03,20190228.0
240,m_c18p_0003,1.181817e+04,20190228.0
241,m_c18p_0006,3.233359e+03,20190228.0
242,m_c18p_0007,1.691577e+04,20190228.0
243,m_c18p_0009,1.154907e+06,20190228.0


# Convert to bigg id

In [7]:
df_bigg = pd.read_csv('Data/han_nature_metabolites_metadata_w_biggids.csv')
df_bigg = df_bigg[df_bigg.universal_bigg_id.notnull()]
df2 = pd.merge(df, df_bigg[['dname','universal_bigg_id']], left_on=['dname'], right_on=['dname'], how='inner')
print("# metabolites in mega medium w/ bigg id = %d"%(len(list(set(df2.dname)))))
df2.head()

# metabolites in mega medium w/ bigg id = 160


,dname,Count,experiment,universal_bigg_id
0,m_c18p_0001,1142.775109,20190228.0,mnl
1,m_c18p_0001,1142.775109,20190228.0,galt
2,m_c18p_0001,4872.896559,20190228.0,mnl
3,m_c18p_0001,4872.896559,20190228.0,galt
4,m_c18p_0001,5370.374879,20190228.0,mnl


# Create mega medium file

In [9]:
mets_in_mm = list(set(df2.universal_bigg_id))
other_supplements = ['cl','fe3','fe2','mn2','mg2','ca2','k','pi','na1','cu2','modb','ni2','zn2','h2co3','h','h2o','so4','cobalt2','no3','sel', # trace minerals
                     'ac','but','ppa','ival', # scfa supplements
                     'cys__L', # L-Cysteine hydrochloride
                     'glc__D','cellb','malt','fru', # carbon sources
                     'his__L', # Histidine-Hematin
                     'mndn', # vitamin K
                     'fol','pydxn','ribflv','btn','thm','nac','pnto__R','cbl1','4abz','lipt' # ATCC Vitamin supplements
                    ]
df_mm = pd.DataFrame(list(set(mets_in_mm+other_supplements))).rename({0:'compound'}, axis=1)
df_mm['medium'] = 'mm'
df_bigg2 = pd.read_csv('Bigg/bigg_models_metabolites.txt', sep='\t')
df_mm = pd.merge(df_mm, df_bigg2[['universal_bigg_id','name']].drop_duplicates().rename({'universal_bigg_id':'compound'}, axis=1), left_on=['compound'], right_on=['compound'], how='left')
df_mm['description'] = 'mega medium, preparation protocol'
df_mm = df_mm[['medium','description','compound','name']]
df_mm.loc[df_mm.compound.isin(mets_in_mm),'description'] = 'mega medium, metabolomics'
df_mm.to_csv('mm.tsv', sep='\t', index=False)
df_mm

,medium,description,compound,name
0,mm,"mega medium, metabolomics",glc__D,D-Glucose
1,mm,"mega medium, metabolomics",ins,Inosine
2,mm,"mega medium, preparation protocol",cobalt2,Co2+
3,mm,"mega medium, preparation protocol",mg2,Magnesium
4,mm,"mega medium, metabolomics",duri,Deoxyuridine
...,...,...,...,...
125,mm,"mega medium, metabolomics",srb__L,L Sorbose C6H12O6
126,mm,"mega medium, metabolomics",ncam,Nicotinamide
127,mm,"mega medium, metabolomics",sucr,Sucrose C12H22O11
128,mm,"mega medium, preparation protocol",lipt,Lipoate C8H13O2S2
